# HW4 Model-based Reinforcement Learning

In [1]:
import sys, os
import tensorflow as tf
import datetime as dt

os.environ['CUDA_VISIBLE_DEVICES'] = '1' # 0 is default GPU

print(tf.__version__)

from tensorflow.python.client import device_lib

local_device_protos = device_lib.list_local_devices()

print(local_device_protos)
print(tf.test.is_gpu_available())
print(tf.test.gpu_device_name())

1.14.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 739271152200791580
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9116851241
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7579671827524831717
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:68:00.0, compute capability: 7.5"
]
True
/device:GPU:0


In [2]:
# main.py

import os
import argparse
import time
import numpy as np

from half_cheetah_env import HalfCheetahEnv
from logger import logger
from model_based_rl import ModelBasedRL

def run_mbrl() :
    parser = argparse.ArgumentParser()
    parser.add_argument('question', type=str, choices=('q1, q2, q3'))
    parser.add_argument('--exp_name', type=str, default=None)
    parser.add_argument('--env', type=str, default='HalfCheetah', choices=('HalfCheetah',))
    parser.add_argument('--render', action='store_true')
    parser.add_argument('--mpc_horizon', type=int, default=15)
    parser.add_argument('--num_random_action_selection', type=int, default=4096)
    parser.add_argument('--nn_layers', type=int, default=1)
    args = parser.parse_args()

    data_dir = os.path.join(os.getcwd(), 'data')
    exp_name = '{0}_{1}_{2}'.format(args.env,
                                    args.question,
                                    args.exp_name if args.exp_name else time.strftime("%d-%m-%Y_%H-%M-%S"))
    exp_dir = os.path.join(data_dir, exp_name)
    assert not os.path.exists(exp_dir),\
        'Experiment directory {0} already exists. Either delete the directory, or run the experiment with a different name'.format(exp_dir)
    os.makedirs(exp_dir, exist_ok=True)
    logger.setup(exp_name, os.path.join(exp_dir, 'log.txt'), 'debug')

    env = {
        'HalfCheetah': HalfCheetahEnv()
    }[args.env]

    mbrl = ModelBasedRL(env=env,
                        render=args.render,
                        mpc_horizon=args.mpc_horizon,
                        num_random_action_selection=args.num_random_action_selection,
                        nn_layers=args.nn_layers,
                        scope='scope-' + str(np.random.rand()))

    run_func = {
        'q1': mbrl.run_q1,
        'q2': mbrl.run_q2,
        'q3': mbrl.run_q3
    }[args.question]
    run_func()
    


## Q1


In [4]:
%tb

sys.argv = 'python main.py q1'.split()[1:]
print('starts running at', dt.datetime.now())
run_mbrl()
print('finished running at', dt.datetime.now())



No traceback available to show.


starts running at 2019-09-23 15:57:07.379697
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


09-23 15:57:07 HalfCheetah_q1_23-09-2019_15-57-07 INFO     Gathering random dataset
I0923 15:57:07.421583 42676 logger.py:107] Gathering random dataset
09-23 15:57:07 HalfCheetah_q1_23-09-2019_15-57-07 INFO     Creating policy
I0923 15:57:07.654002 42676 logger.py:107] Creating policy


ValueError: Variable model/dense/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 2005, in __init__
    self._traceback = tf_stack.extract_stack()
  File "C:\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 3616, in create_op
    op_def=op_def)
  File "C:\Anaconda3\lib\site-packages\tensorflow\python\util\deprecation.py", line 507, in new_func
    return func(*args, **kwargs)
  File "C:\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 788, in _apply_op_helper
    op_def=op_def)
  File "C:\Anaconda3\lib\site-packages\tensorflow\python\ops\gen_state_ops.py", line 2024, in variable_v2
    shared_name=shared_name, name=name)
